# 📊 **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)

I want to start my notebook by acknowledging that I used epsilon of 1 when calling the standard scaler. Other parts are pretty similar to the 1st Trial Notebook. Initially, I did not know that each participant only has 10 epsilon in total for this competition. For anyone in the future, be mindful with calling the functions including epsilon as their parameters.


### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 254a5103-244e-41d5-8a9d-88bbc758bb43, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

In [ ]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+



### 🎈 Import packages

Please make sure to read [Antigranular DOCS](https://docs.antigranular.com/) for all the packages and functions that you are importing.

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



### 🎈 A simply solution

The model itself is slightly different from the model in 1st Trial Notebook. I simply increased the number of nodes in the neural networks and decreased the dropout ratio.

In [ ]:
%%ag
# Normal keras model
seqM = Sequential([
    Dense(128, activation='relu', input_shape=(6,)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

***Warning***

Make sure not to call the standard_scaler function with high epsilon value! This will cap out your total epsilon very quickly. Test with small epsilon value all the time. Also

In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=1)
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=1)

In order to train your data in the model, you need to load it using the PrivateDataLoader function.

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=64)

### 🎈 Training

Before training, review [Tensorflow Privacy](https://docs.antigranular.com/private-python/packages/tensorflow) and calculate the epsilon required, since fit function can use a lot of your precious epsilon.

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=100, target_delta=1e-5)

Epoch 1/100

125/125 - 5s - loss: 0.4794 - accuracy: 0.7562 - 5s/epoch - 43ms/step

Epoch 2/100

125/125 - 2s - loss: 0.3815 - accuracy: 0.8151 - 2s/epoch - 18ms/step

Epoch 3/100

125/125 - 2s - loss: 0.3250 - accuracy: 0.8469 - 2s/epoch - 19ms/step

Epoch 4/100

125/125 - 2s - loss: 0.2713 - accuracy: 0.8829 - 2s/epoch - 18ms/step

Epoch 5/100

125/125 - 2s - loss: 0.2314 - accuracy: 0.8956 - 2s/epoch - 18ms/step

Epoch 6/100

125/125 - 2s - loss: 0.1960 - accuracy: 0.9185 - 2s/epoch - 18ms/step

Epoch 7/100

125/125 - 2s - loss: 0.1942 - accuracy: 0.9188 - 2s/epoch - 18ms/step

Epoch 8/100

125/125 - 2s - loss: 0.1793 - accuracy: 0.9267 - 2s/epoch - 18ms/step

Epoch 9/100

125/125 - 2s - loss: 0.1641 - accuracy: 0.9335 - 2s/epoch - 18ms/step

Epoch 10/100

125/125 - 2s - loss: 0.1487 - accuracy: 0.9440 - 2s/epoch - 18ms/step

Epoch 11/100

125/125 - 2s - loss: 0.1452 - accuracy: 0.9406 - 2s/epoch - 18ms/step

Epoch 12/100

125/125 - 2s - loss: 0.1283 - accuracy: 0.9502 - 2s/epoch - 

If you scaled your training data, make sure to scale your test data before you predict.

In [ ]:
%%ag
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 1s 6ms/step



If you have 2 nodes in the last layer except 1 in your neural network, then you do not need to do this, but since I have only 1 node in the last layer, I need to divide my prediction into either 1 or 0.

In [ ]:
%%ag
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

### 📝 Make your submission (Through AG)

Submit a prediction by simply typing `submit_predictions(your_prediction)` to find out how you rank on the leaderboard.

![img](https://www.antigranular.com/static/media/Step%209.8091828f3cff4324fe6d.png)


In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.8425859130351336, 'logs': {'BIN_ACC': 0.8545321030863583, 'LIN_EPS': -0.011946190051224609}}



### Improvement

1. Based on Omer Yentur's [solution](https://www.antigranular.com/notebooks/66a1f94c019ee6315e603829), Omer used own scaler instead of the standard_scaler function. Using standard_scaler function introduces randomness based on the value of epsilon (the lower epsilon value gives more randomness). Hence, implementing your own scaler will increase the performance dramatically as Omer Yentur did.
2. The cutoff for predictions was 0.5 for this notebook. There may be better cutoff value than 0.5.
3. More layers will result in better performance, as Mosbeh Barhoumi's [solution](https://www.antigranular.com/notebooks/66a2118a019ee6315e62023e) and Ala Ammari's [solution](https://www.antigranular.com/notebooks/66a21447019ee6315e6239ce) show. They both used epsilon value of 0.9, lower than mine, and used neural network with more layers.



